In [ ]:
%pip install -q --upgrade keras-cv

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import sys
import os
import keras_cv

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

directory = "/content/"
user_data = directory + "training_data"
test_data = directory + "label_book/" # this can be the label book, or any other test set you create

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/10LEPW3Di4dhqy_NASZ33G0eagi4w3CS4/view?usp=sharing # Replace the link with your file

In [ ]:
!unzip Final.zip

# Rules

- Submission must have less than 12,000 images combined in training and validation

**Submissions will be evaluated according to two categories:**
1. **Best Performance Overall**
2. **Most Innovative**

# Getting started


The dataset contains ~4400 images of handwritten roman numerals 1-10. **Your task is to optimize model performance by improving the dataset and making training and validation splits.**

You can try fixing incorrect labels, adding data for side case tuning, apply data augmentation techniques, or use any other method to improve the data. You may also find it helpful to take a look at the training script to get a better sense of the model below (these are held fixed). The script will resize all images to `(32, 32)` and run them through a cut off version of ResNet50.

In [ ]:
### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
tf.keras.backend.clear_session()
batch_size = 8
tf.random.set_seed(2024)

train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

valid = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
)

total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()

if total_length > 12_000:
    print(f"Dataset size larger than 12,000. Got {total_length} examples")
    sys.exit()

test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
)

# Initialize the base model using KerasCV's ResNet50
backbone = keras_cv.models.ResNet50Backbone.from_preset(
    input_shape=(32, 32, 3),
    preset = "resnet50_imagenet",
    load_weights=False,
)

# Create a new model that outputs the desired intermediate layer
base_model = tf.keras.Model(
    inputs=backbone.inputs,
    outputs=backbone.get_layer("v2_stack_0_block3_out").output
)

# Define the input tensor
inputs = tf.keras.Input(shape=(32, 32, 3))

# Pass the preprocessed input through the base model
x = base_model(inputs)

# Add global average pooling
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Add a dense layer for classification (assuming 10 classes)
x = tf.keras.layers.Dense(10)(x)

# Define the final model
model = tf.keras.Model(inputs, x)

# Compile the model with appropriate optimizer, loss, and metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Display the model's architecture
model.summary()

loss_0, acc_0 = model.evaluate(valid)
print(f"loss {loss_0}, acc {acc_0}")

checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model.weights.h5",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, verbose=1, min_lr=1e-7)

history = model.fit(
        train,
        validation_data=valid,
        epochs=75,
        callbacks=[checkpoint, lr_scheduler],
)

model.load_weights("best_model.weights.h5")

loss, acc = model.evaluate(valid)
print(f"final loss {loss}, final acc {acc}")

test_loss, test_acc = model.evaluate(test)
print(f"test loss {test_loss}, test acc {test_acc}")

### DO NOT MODIFY ABOVE THIS LINE, THIS IS THE FIXED MODEL ###

## Make your submission

Remember that the evaluation metric is Macro F1 score, you may want to also evaluate the above results using the Macro F1 score before submission.

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory+"testing_data",
    shuffle = False,
    image_size=(32, 32),
    batch_size=1)

prob = model.predict(test_dataset)
predictions = []
for i in range(0, prob.shape[0]):
    predictions.append(np.argmax(prob[i,:])+1)

In [ ]:
import pandas as pd

paths = test_dataset.file_paths

Ids = []
for x in paths:
    Ids.append(x.split("/")[-1])

df = pd.DataFrame()
df["Id"] = Ids
df["Predicted"] = predictions
df.to_csv("submission.csv", index=False)